In [2]:
%matplotlib

import math 
import pywt 
import numpy as np
import os
import csv
import fnmatch
import datetime
import re 
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats

Using matplotlib backend: Qt5Agg


In [6]:
RemoveNtrials = 2

In [7]:
TimeDwellOrig = 800
TimeFixation = 300

In [8]:
def filterBlinks(pupilData, timeListComplete):
    # filter any blinks and nan values lasting around 250ms (on average)
    # http://faculty.washington.edu/chudler/facts.html
   
    
   # Blink detection    
    missingPupilData = dict()
    missingPupilData['Start'] = list()
    missingPupilData['End'] = list()
    missingPupilData['Duration'] = list()
    missingPupilData['DurationExtra'] = list()
    
    pupilData_filter = list()
    time_filter = list()
    index_blinkEnd = list()
    
    missingBeginning = 0
    missingEnd = 0

    # First, find the missing values
    pupilInd = -1
    for pupilD in pupilData:
        
        pupilInd = pupilInd + 1
        #if pupilInd>772 and pupilInd<940:
        #    print(pupilInd, pupilD)
            
        # check if val is nan
        if np.isnan(pupilD):
            # check if first element of list
            if pupilInd == 0:
                # first value is nan
                missingPupilData['Start'].append(pupilInd)
                #print('Start', missingPupilData['Start'][-1])
                
            # check if previous val was nan
            elif np.isnan(pupilData[pupilInd-1]):
                
                # check if val is not last element
                if pupilInd < len(pupilData)-1:
                    # check if next val is not nan
                    if not np.isnan(pupilData[pupilInd+1]):
                        # note the index for end of blink
                        missingPupilData['End'].append(pupilInd)
                        #print('End', missingPupilData['End'][-1])
                else:
                    # last value is nan
                    missingEnd = 1
                    missingPupilData['End'].append(pupilInd)
                    #print('End', missingPupilData['End'][-1])
                    
            # if new nan, note the index
            else:
                missingPupilData['Start'].append(pupilInd)
                #print('Start', missingPupilData['Start'][-1])
                
                # for single nan values
                # check if val is not last element
                if pupilInd < len(pupilData)-1:
                    # check if next val is not nan
                    if not np.isnan(pupilData[pupilInd+1]):
                        # note the index for end of blink
                        missingPupilData['End'].append(pupilInd)
                        #print('End', missingPupilData['End'][-1])
                else:
                    # last value is nan
                    missingPupilData['End'].append(pupilInd)
                    #print('End', missingPupilData['End'][-1])
            
    
    
    # Second, create a list that does not contain missing values or 200ms before and after that
    
     # blink is every nan value in the range of 100-400ms 
    # 200 ms (18 samples) before and after the blink will also be removed
    addedBlinkSamples = 18    
    blinkDurationStart = 0
    blinkDurationEnd = 0
    
    iCurrent = len(missingPupilData['Start'])
    endCurrent = iCurrent - 1
    isBlink = 0
    pupilData_filter = list(pupilData)
    time_filter = list(timeListComplete)
    
    index_blinkEndExtra = list() # list to document if 2 blinks joined 
    
    while iCurrent > 0:
        
        iCurrent = iCurrent - 1
        
        #print('Current', missingPupilData['Start'][iCurrent], missingPupilData['End'][iCurrent])
        
        # check if it is really a blink, and remove if it is not
        # if blink duration is less than 100ms, the reason for missing data is something else
        timeStart0 = time_filter[missingPupilData['Start'][iCurrent]].partition('+')
        timeStart = (datetime.datetime.strptime(re.sub('[:.T]','-',timeStart0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                
        timeEnd0 = time_filter[missingPupilData['End'][iCurrent]].partition('+')
        timeEnd = (datetime.datetime.strptime(re.sub('[:.T]','-',timeEnd0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                
        time1 = (timeEnd - timeStart).total_seconds()
        #print(time1)
        if time1 < 0.1 or time1 > 1.5:
            isNotBlink = 1
        else:
            isNotBlink = 0
            
        # Remove if it is a blink
        if isNotBlink:
            del pupilData_filter[missingPupilData['Start'][iCurrent]:missingPupilData['End'][iCurrent]+1]
            del time_filter[missingPupilData['Start'][iCurrent]:missingPupilData['End'][iCurrent]+1]
            
            # length of non-blink
            lengthBlink = missingPupilData['End'][iCurrent] - missingPupilData['Start'][iCurrent] + 1
            
            # all blinks after this non-blink need to be reduced in indexes
            #print(len(missingPupilData['Start'])-1)
            
            for iChange in range(iCurrent-1, len(missingPupilData['Start'])-1):
                #print('i', iChange)
                if iChange == iCurrent-1:
                    
                    #print('Deleted', missingPupilData['Start'][iChange+1], missingPupilData['End'][iChange+1])
                    #print(lengthBlink)
                    del missingPupilData['Start'][iChange+1]
                    del missingPupilData['End'][iChange+1]
                else:
                    
                    #print('Old', missingPupilData['Start'][iChange], missingPupilData['End'][iChange])
                    missingPupilData['Start'][iChange] = missingPupilData['Start'][iChange] - lengthBlink
                    missingPupilData['End'][iChange] = missingPupilData['End'][iChange] - lengthBlink
                    #print('New', missingPupilData['Start'][iChange], missingPupilData['End'][iChange])
                    
    #print(len(missingPupilData['Start']))
    
    previousBlinkEnd = 0
    nextBlinkEnd = 0
    iCurrent = len(missingPupilData['Start'])
    
#     for index, (value1, value2) in enumerate(zip(missingPupilData['Start'], missingPupilData['End'])):
#         print(index, value1, value2)
#     for i in enumerate(pupilData_filter):
#         print(i)
    
    while iCurrent > 0:
        
        iCurrent = iCurrent - 1
        #print(iCurrent)
        iNow = iCurrent # iCurrent is modified in advance later (possibly), which will disturb the current data index
        #print(missingPupilData['Start'][iNow], missingPupilData['End'][iNow])
        
#         if iNow > 1 :
#             print('BEFORE!!!!!!!!!!!!!!!!!')
#             print(pupilData_filter[missingPupilData['Start'][iNow]:-1])
            
        # to make sure that blink removal data does not go into the previous data
        if iCurrent == 0:
            previousBlinkEnd = 0
        else:
            previousBlinkEnd = missingPupilData['Start'][iCurrent-1]

        # to make sure blink does not go into the next one
        if iCurrent == len(missingPupilData['Start'])-1:
            #print(iCurrent)
            nextBlinkEnd = len(pupilData_filter)
            #print(nextBlinkEnd)
        else:
            nextBlinkEnd = missingPupilData['Start'][iCurrent+1]
            
        
        
        if missingPupilData['End'][iCurrent] + addedBlinkSamples > nextBlinkEnd:
            #LOOK INTO THIS, NEXT BLINK IS REMOVED BUT ITS EXTRA 200MS IS NOT!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            #print('After', missingPupilData['End'][iCurrent], addedBlinkSamples, '>>>>>', nextBlinkEnd)
            blinkRemoveAfter = abs(missingPupilData['End'][iCurrent] - nextBlinkEnd)
        else:
            blinkRemoveAfter = addedBlinkSamples
        
        if missingPupilData['Start'][iCurrent] - addedBlinkSamples < previousBlinkEnd:
            #print('Before', missingPupilData['Start'][iCurrent], addedBlinkSamples, '<<<<<<', previousBlinkEnd)
            blinkRemoveBefore = abs(missingPupilData['Start'][iCurrent] - previousBlinkEnd)
            
            # Add the extra duration to list
            if iCurrent > 0:
                
                timeStart0 = time_filter[missingPupilData['Start'][iCurrent-1]].partition('+')
                timeStart = (datetime.datetime.strptime(re.sub('[:.T]','-',timeStart0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                
                timeEnd0 = time_filter[missingPupilData['End'][iCurrent-1]].partition('+')
                timeEnd = (datetime.datetime.strptime(re.sub('[:.T]','-',timeEnd0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                
                time1 = (timeEnd - timeStart).total_seconds()
                #print(timeEnd, timeStart, time1)

                missingPupilData['DurationExtra'].append(time1)
#             if iCurrent > 1:
#                 print(missingPupilData['Start'][iCurrent-1], 'also removed')
#             # After next blink is also removed, 
            iCurrent = iCurrent - 1
            index_blinkEndExtra.append(2)
            
                
        else:
            blinkRemoveBefore = addedBlinkSamples
            index_blinkEndExtra.append(1)
            missingPupilData['DurationExtra'].append(0)
            
#         print(missingPupilData['Start'][iNow]-blinkRemoveBefore)
        
        timeStart0 = time_filter[missingPupilData['Start'][iNow]].partition('+')
        timeStart = (datetime.datetime.strptime(re.sub('[:.T]','-',timeStart0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
        #print(timeStart, pupilData_filter[missingPupilData['Start'][iNow]])     
        timeEnd0 = time_filter[missingPupilData['End'][iNow]].partition('+')
        timeEnd = (datetime.datetime.strptime(re.sub('[:.T]','-',timeEnd0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
        #print(timeEnd) 
        time2 = (timeEnd - timeStart).total_seconds()
        #print(time2)
        missingPupilData['Duration'].append(time2)
        del pupilData_filter[missingPupilData['Start'][iNow]-blinkRemoveBefore:missingPupilData['End'][iNow]+blinkRemoveAfter]
        del time_filter[missingPupilData['Start'][iNow]-blinkRemoveBefore:missingPupilData['End'][iNow]+blinkRemoveAfter]
        
        index_blinkEnd.append(missingPupilData['Start'][iNow] - blinkRemoveBefore - 1)
    #         if iNow > 1 :
#             print('AFTER!!!!!!!!!!!!')
#             if np.nan in pupilData_filter[missingPupilData['Start'][iNow]:-1]:
#                 for i,j in enumerate(pupilData_filter[missingPupilData['Start'][iNow]:-1]):
#                     print(i+missingPupilData['End'][iNow]+blinkRemoveAfter,j)
        
    
    if np.nan in pupilData_filter:
        for i in enumerate(pupilData_filter):
            print(i)
     
    index_blinkEnd = list(reversed(index_blinkEnd))
    
    return pupilData_filter, time_filter, index_blinkEnd, index_blinkEndExtra, missingPupilData

In [9]:
def ComputeDwellTime(userKeys):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        if key[1] == 'IncreaseTimeDwell':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseTimeDwell':
            if float(key[2]) == 1:
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [28]:
def UserLookedAtNewTrial(UserLooksAtKey):
    # Create list of list (made of epochs) composed of times when user looked at NextPhrase
    
    epoch = list()   
    epochList = list()
    
    nRow = -1
    firstNextPhrase = 0
    flagStart = 0
    firstTrial = 0
    
    # Find unique looks at new trial, by checking if progress pct is greater than previous one and store first look in a list
    
    progressNextPhrase = 0 
    
    for row in UserLooksAtKey:
        nRow = nRow + 1
        
        # first add data for first trial, before first next phrase is reached
        if firstTrial == 0:
            firstTrial = 1
            epochList.append([row, 0])
            ##print(epochList)
            continue
            
        if row[1] == 'NextPhrase':
            
            # first eliminate an ongoing look
            if UserLooksAtKey[nRow-1][1] == 'NextPhrase':
                continue
            else:
                # normally for new looks
                
                # if it is a continued look, progressNextPhrase should not be 0
                ##print('Continued ???', progressNextPhrase, UserLooksAtKey[nRow])
                
                if progressNextPhrase > float(UserLooksAtKey[nRow][2]):
                    
                    # save it as 0, if not a continued look
                    ##print('progress is now 0')
                    progressNextPhrase = 0
#                 else:
#                     if progressNextPhrase == float(UserLooksAtKey[nRow][2]):
#                         #print('ITS EQUAL!!!')
                    
                
            # now check if this look is going to be successfully selected:
            nextPhrase_selected = 0
            rowCurrent = nRow-1
            
            while nextPhrase_selected < 1:
                rowCurrent = rowCurrent + 1
                if UserLooksAtKey[rowCurrent][1] == 'NextPhrase':
                    if float(UserLooksAtKey[rowCurrent][2]) == 1:
                        ##print(UserLooksAtKey[rowCurrent])
                        nextPhrase_selected = 1
                        rowCurrentEnd = rowCurrent
                        
                        
                        if progressNextPhrase != 0:
                            epochList[-1].insert(2, UserLooksAtKey[rowPreviousEnd])
                            epochList.append([UserLooksAtKey[rowCurrentStart], UserLooksAtKey[rowCurrentEnd]])
                        else:
                            epochList[-1].insert(2, UserLooksAtKey[nRow-1])
                            epochList.append([UserLooksAtKey[nRow], UserLooksAtKey[rowCurrentEnd]])
                    else:
                        continue
                else:
                    # if look at next phrase is stopped, store the previous look progress, in case it is continued later
                    progressNextPhrase = float(UserLooksAtKey[rowCurrent-1][2])
                    rowCurrentStart = nRow
                    rowPreviousEnd = nRow - 1
                    break
                    
                     
                    
                    
    # insert last element before quiting - 
    # keys looked at list
    KeysLookedAtFromUserKeys = [key[1] for key in UserLooksAtKey]
    
    # find key before quit key
    nBegin = 300
    foundQuitKey = 0
    indBeforeQuit = 0
    while foundQuitKey < 1:
        if KeysLookedAtFromUserKeys[-nBegin] == 'Quit':
            # look in direction of previous keys
            # if previous key is not quit, you can stop
            if KeysLookedAtFromUserKeys[-nBegin-1] != 'Quit':
                foundQuitKey = 1
                indBeforeQuit = -nBegin - 1
                break
            else:
                nBegin = nBegin + 1
        else:
            # current key is not quit, start looking in the other direction
            if KeysLookedAtFromUserKeys[-nBegin + 1] == 'Quit':
                foundQuitKey = 1
                indBeforeQuit = -nBegin
                break
            else:
                nBegin = nBegin - 1
    
    epochList[-1].insert(2, UserLooksAtKey[indBeforeQuit])
    
    iRemove = RemoveNtrials
    while iRemove > 0:
        iRemove = iRemove - 1
        epochList.remove(epochList[iRemove])
    
#     for i in epochList:
#         print(i)
#     print('next')
    return epochList[0][0][0]

In [19]:
# function to convert list of date and time into datetime format list

def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [20]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [27]:
# find end time of Expt - when the scratchpad is erased, at the end
def FindTimeEnd(scratchPadList):
    
    scratchPadText = [i[1] for i in scratchPadList]
    scratchPadTime = [i[0] for i in scratchPadList]
                
    iText = len(scratchPadText)
    while iText > 0:
        iText = iText - 1
        if scratchPadText[iText] == '':
            if scratchPadText[iText-1] != '':
                findTimeAtIndex = iText
                break
                    
    return scratchPadTime[findTimeAtIndex]

In [30]:
subjName = r'C:\DTU\Data\201805_HealthnRehab\TypingData'
j = 0
flagFirstSubj = 0
ipaList = list()
blink = dict()
blink['rate_DT'] = list()
blink['rate_MS'] = list()

blink['duration_DT'] = list()
blink['duration_MS'] = list()

blink['durationRatio_DT'] = list()
blink['durationRatio_MS'] = list()

for root, dirs, subfolder in os.walk(subjName):
    LetterLookedAtList = list()
    LetterLookedAt = list()
    
    if not dirs:
        
        if 'notCompleted' in root or 'notInclude' in root or 'notEnough' in root: # Some subjects do not have gaze log and have been marked as 
            #notCompleted, or they were not native Danish speakers and are marked as notInclude
            continue
        if 'tb' in root or 'joha' in root:
            continue
            
        userKeys = None
        scratchPad = None
        gazeLog = None
        stimPhrase = None
        
        for file in subfolder:
            
            
            if fnmatch.fnmatch(file, 'GazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8')
                    readerGazeLog = csv.reader(fGazeLog)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0]) # would not matter much even if the first row was not labels
                    gazeLog.remove(gazeLog[-1])
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the gaze log file')
            elif fnmatch.fnmatch(file, 'user_look*'):
                try:
                    
                    fUserKey = open(root + '\\' + file, encoding='utf-8')
                    readerUserKey = csv.reader(fUserKey)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user looks at log file')
                        
            elif fnmatch.fnmatch(file, 'ScratchPad*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8')
                    readerScratchPad = csv.reader(fScratchPad)
                    scratchPad = list(readerScratchPad)  
                    scratchPad.remove(scratchPad[0])
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the user looks at log file')
            else:
                continue
            
                # if all these lists exist
            if gazeLog is None or scratchPad is None or userKeys is None:
                continue
            else:
                
                a = re.compile('(?<=TypingData\\\May[0-9]{2}\\\)(.*)(?=\\\OptiKey)')
                subjName = a.findall(root)[0]
                typing_mechanism = subjName[-2:]
                print(subjName)
                
                # obtain right and left pupil data
                # Create list of pupil sizes from gazelog
                pupilLogL = [float(item4[29]) if 'Invalid' not in item4 else np.nan for item4 in gazeLog]
                pupilLogR = [float(item5[31]) if 'Invalid' not in item5 else np.nan for item5 in gazeLog]
                timeList = [item[0] for item in gazeLog]
                
                # filter the blinks
                pupilLogL_filter, time_filter, index_blinkEndL, index_blinkEndExtra, missingDataL = filterBlinks(pupilLogL, timeList)
                
                pupilLogR_filter, time_filter, index_blinkEndR, index_blinkEndExtra, missingDataR = filterBlinks(pupilLogR, timeList)
                
                time_filterFormatted = timeConversion(time_filter)
                pupilLog_avg = [(pupilLogL_filter[i] + pupilLogR_filter[i])/2 for i in range(0, len(pupilLogL_filter))]
                
                # Compute dwell time
                userKeysWithTime = ComputeDwellTime(userKeys)

                # call function to check when scratchpad is looked at and save it in a list
                timeExptStartStr = UserLookedAtNewTrial(userKeysWithTime)
                timeExptStartStr1, t1, t2 = timeExptStartStr.partition('+')
                timeExptStart = (datetime.datetime.strptime(re.sub('[:.T]','-',timeExptStartStr1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                
                
                timeExptEndStr = FindTimeEnd(scratchPad)
                timeExptEndStr1, t1, t2 = timeExptEndStr.partition('+')
                timeExptEnd = (datetime.datetime.strptime(re.sub('[:.T]','-',timeExptEndStr1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                
                # find the actual start of the fixation
        
                timePeriodExpt = ((timeExptEnd - timeExptStart).total_seconds())/60
                blinkRate = len(index_blinkEndL)/(timePeriodExpt)/60
                
                blinkDuration = sum(missingDataL['Duration'])
                blinkToExptRatio = blinkDuration/timePeriodExpt
                
                if 'DT' in subjName:
                    blink['rate_DT'].append(blinkRate)
                    blink['duration_DT'].append(blinkDuration)
                    blink['durationRatio_DT'].append(blinkToExptRatio)
                else:
                    blink['rate_MS'].append(blinkRate)
                    blink['duration_MS'].append(blinkDuration)
                    blink['durationRatio_MS'].append(blinkToExptRatio)
                
                
                print(blinkDuration)
                print(blinkRate)


akt_MS
74.09299700000004
0.32236989213037126
KEA_MS
3.848982
0.09705150067969533
mcc_MS
20.756942000000002
0.16161455363297647
MK_DT
15.397209
0.2086771051507618
ok_MS
11.940587
0.11530610966333614
pt_DT
11.363030000000004
0.20309863637324987
sh_MS
21.961403000000004
0.24660348728210063
hc_MS
35.566066
0.28772243991666113
ib_MS
81.54202399999994
0.3057238975296411
km_DT
7.651732000000001
0.06129288546356323
ma_DT
10.990131000000002
0.2299613327377764
pgba_DT
1.070431
0.01059564858327808
smn_DT
14.212274999999996
0.08567372951917479
jek_MS
34.023520000000005
0.20006389055734478
lr_MS
2.1087909999999996
0.03721425845705653
mm_MS
20.18743
0.3485392754611486
snk_DT
16.962249999999997
0.17253867758504884


In [17]:
# Boxplot

fig = plt.figure()
ax = fig.add_subplot(1,1,1) 

ax.boxplot([blink['rate_DT'], blink['rate_MS']], positions= [1, 2])
xTicks = ax.get_xticks()
#print(xTicks)
ax.set_xticklabels(['DT', 'MS'])

ax.set_title('DT- mean: {0}, std: {1}                      MS - mean:  {2},  std:  {3}'.format(np.round(np.mean(blink['rate_DT']), 3), np.round(np.std(blink['rate_DT']),3), np.round(np.mean(blink['rate_MS']),3), np.round(np.std(blink['rate_MS']),3)))
#ax.set_ylim([0, 0.5])

print(scipy.stats.ttest_ind(blink['rate_DT'], blink['rate_MS']))
print(scipy.stats.ranksums(blink['rate_DT'], blink['rate_MS']))

print('DT- mean: {0}, std: {1}                      MS - mean:  {2},  std:  {3}'.format(np.round(np.mean(blink['rate_DT']), 3), np.round(np.std(blink['rate_DT']),3), np.round(np.mean(blink['rate_MS']),3), np.round(np.std(blink['rate_MS']),3)))

Ttest_indResult(statistic=-1.5090900015122577, pvalue=0.15204958081993447)
RanksumsResult(statistic=-1.3662601021279466, pvalue=0.1718573390627992)
DT- mean: 8.33, std: 4.742                      MS - mean:  12.733,  std:  6.07


In [24]:
# Boxplot

fig = plt.figure()
ax = fig.add_subplot(1,1,1) 

ax.boxplot([blink['duration_DT'], blink['duration_MS']], positions= [1, 2])
xTicks = ax.get_xticks()
#print(xTicks)
ax.set_xticklabels(['DT', 'MS'])

ax.set_title('DT- mean: {0}, std: {1}                      MS - mean:  {2},  std:  {3}'.format(np.round(np.mean(blink['duration_DT']), 3), np.round(np.std(blink['duration_DT']),3), np.round(np.mean(blink['duration_MS']),3), np.round(np.std(blink['duration_MS']),3)))
#ax.set_ylim([0, 0.5])

print(scipy.stats.ttest_ind(blink['duration_DT'], blink['duration_MS']))
print(scipy.stats.ranksums(blink['duration_DT'], blink['duration_MS']))

print('DT- mean: {0}, std: {1}                      MS - mean:  {2},  std:  {3}'.format(np.round(np.mean(blink['duration_DT']), 3), np.round(np.std(blink['duration_DT']),3), np.round(np.mean(blink['rate_MS']),3), np.round(np.std(blink['rate_MS']),3)))

Ttest_indResult(statistic=-1.8521480117392488, pvalue=0.08378704819517226)
RanksumsResult(statistic=-1.9518001458970664, pvalue=0.050961936967763306)
DT- mean: 11.092, std: 5.001                      MS - mean:  12.733,  std:  6.07


In [26]:
# Boxplot

fig = plt.figure()
ax = fig.add_subplot(1,1,1) 

ax.boxplot([blink['durationRatio_DT'], blink['durationRatio_MS']], positions= [1, 2])
xTicks = ax.get_xticks()
#print(xTicks)
ax.set_xticklabels(['DT', 'MS'])

ax.set_title('DT- mean: {0}, std: {1}                      MS - mean:  {2},  std:  {3}'.format(np.round(np.mean(blink['durationRatio_DT']), 3), np.round(np.std(blink['durationRatio_DT']),3), np.round(np.mean(blink['durationRatio_MS']),3), np.round(np.std(blink['durationRatio_MS']),3)))
#ax.set_ylim([0, 0.5])

print(scipy.stats.ttest_ind(blink['durationRatio_DT'], blink['durationRatio_MS']))
print(scipy.stats.ranksums(blink['durationRatio_DT'], blink['durationRatio_MS']))

print('DT- mean: {0}, std: {1}                      MS - mean:  {2},  std:  {3}'.format(np.round(np.mean(blink['durationRatio_DT']), 3), np.round(np.std(blink['durationRatio_DT']),3), np.round(np.mean(blink['rate_MS']),3), np.round(np.std(blink['rate_MS']),3)))

Ttest_indResult(statistic=-2.2725604216580373, pvalue=0.03819394690683467)
RanksumsResult(statistic=-2.3421601750764798, pvalue=0.01917248475522309)
DT- mean: 3.211, std: 1.639                      MS - mean:  12.733,  std:  6.07
